In [ ]:
import os
import sys
import json
import math
from astropy.time import Time as astrotime
from tqdm import tqdm_notebook as tqdm

import numpy as np
from numpy.lib.recfunctions import drop_fields

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline

import scipy.interpolate as sci
import scipy.optimize as sco
import scipy.integrate as scint
import scipy.stats as scs

import anapymods3.plots.astro as amp_plt
from anapymods3.plots import (split_axis, get_binmids, hist_marginalize,
                              hist_from_counts, dg)
from anapymods3.stats import rejection_sampling, json2kde, sigma2prob
from anapymods3.general import fill_dict_defaults
from anapymods3.healpy import wrap_theta_phi_range

import tdepps.bg_injector as BGInj
import tdepps.bg_rate_injector as BGRateInj
import tdepps.rate_function as RateFunc
import tdepps.llh as LLH
import tdepps.analysis as Analysis
import tdepps.signal_injector as SigInj
from tdepps.utils import rejection_sampling, func_min_in_interval, rotator

secinday = 24. * 60. * 60.
print("Executed on ", astrotime.now())

Test multi year analysis build on top of the single year one.

# Setup GRB sample

In [ ]:
rndgen = np.random.RandomState(7353)

In [ ]:
data = np.load("data/GRB_IC86III/data.npy")
sim = np.load("data/GRB_IC86III/sim_IC86III_nugen_10634.npy")
grblist = np.load("data/GRB_IC86III/grblist.npy")
with open("data/GRB_IC86III/runlist.json", "r") as infile:
    runlist = json.load(infile)
    
nfiles = 956
nevents = 5000
ngen = float(nfiles * nevents)
    
# Cut on score
score_cut = 0.5
data = data[data["score"] > score_cut]
sim = sim[sim["score"] > score_cut]
    
run_ids = runlist.keys()
livetimes = []
for key in run_ids:
    livetimes.append(runlist[key]["livetime"])
livetimes = np.array(livetimes)
livetime = np.sum(livetimes)
livetime_days = livetime / 24. / 3600.

In [ ]:
def is_on_time(run_id):
    key = str(int(run_id))
    return True if runlist[key]["ontime"] else False

on_mask = np.array(list(map(is_on_time, data["run_id"])))
assert np.sum(on_mask) + np.sum(~on_mask) == len(data)

on_data = data[on_mask]
off_data = data[~on_mask]

# Get livetime split
on_livetime = np.sum([run["livetime"] if run["ontime"] else 0.
                       for run in runlist.values()])
off_livetime = np.sum([run["livetime"] if not run["ontime"] else 0.
                       for run in runlist.values()])
assert livetime == (on_livetime + off_livetime)

print("Off livetime   : {:6.2f} days".format(off_livetime / 24. / 3600.))
print("On livetime    : {:6.2f} days".format(on_livetime / 24. / 3600.))
print("Total livetime : {:6.2f} days".format(livetime_days))

In [ ]:
nsrcs = len(grblist)
names = ["t", "dt0", "dt1", "ra", "dec", "w_theo", "sigma"]
types = len(names) * [np.float]
dtype = [(name, typ) for name, typ in zip(names, types)]

grb_srcs = np.empty((nsrcs, ), dtype=dtype)

grb_srcs["t"] = grblist["timeMJD"]
grb_srcs["dt0"] = np.zeros(nsrcs, dtype=np.float)
grb_srcs["dt1"] = grblist["t100"]
grb_srcs["ra"] = grblist["ra"]
grb_srcs["dec"] = grblist["dec"]
grb_srcs["w_theo"] = np.ones(nsrcs, dtype=np.float)
grb_srcs["sigma"] = grblist["sigma"]

In [ ]:
runs = []
for key, item in runlist.items():
    d = item.copy()
    tstart = astrotime(item["tStart"], format="mjd").iso
    tstop = astrotime(item["tStop"], format="mjd").iso
    d["run"] = int(key)
    d["good_tstart"] = tstart
    d["good_tstop"] = tstop
    runs.append(d)

goodruns = {"runs": runs}
print("Number of all runs : ", len(runs))
off_len =  len(filter(lambda runi: not runi["ontime"], runs))
on_len =  len(filter(lambda runi: runi["ontime"], runs))
print("Number of off runs : ",off_len)
print("Number of on runs  : ", on_len)

assert off_len + on_len == len(runs)

In [ ]:
rndgen = np.random.RandomState(28756)

# Create the GRBLLH object with all the PDF settings
# Note: Range derived from sinDec hist in NRT wiki
sin_dec_bins = np.linspace(-0.1, 1, 17 + 1)  

# Range is derived from hist, dropping some wrongly reconstructed events
min_logE = -1.
max_logE = 10.
logE_bins = np.linspace(min_logE, max_logE, 40)

# Remove wrongly reconstructed events outside range from off data & sim
logE_mask = (sim["logE"] > min_logE) & (sim["logE"] < max_logE)
sim_pdf_full = sim[logE_mask]
print("Removed {} events in logE for simulation.".format(np.sum(~logE_mask)))
logE_mask = (off_data["logE"] > min_logE) & (off_data["logE"] < max_logE)
data_pdf_full = off_data[logE_mask]
print("Removed {} events in logE for data.".format(np.sum(~logE_mask)))

# Later 3 events show up, that have times outside the run times. These are
# removed too - still strange.
time_idx = [39023, 52880, 63524]
data_pdf_full = np.delete(data_pdf_full, time_idx)

# Check weights for E2 signal against built-in weights
gamma = 2.
type_w = 0.5         # Nugen defaults: 50:50 nu, anti-nu
norm_per_type = 0.5  # Astro just splits 50:50
sim_pdf_full["ow"] /= ngen * type_w  # Norm to OW per type and "sim-livetime"
w = norm_per_type * sim_pdf_full["trueE"]**(-gamma) * sim_pdf_full["ow"]

print("Included and self made weights ar the same: ",
      np.allclose(sim_pdf_full["weight_E2"], w))

# Now strip off uneeded fields so data and sim are compatible for tdepps
data_pdf = drop_fields(data_pdf_full, ["run_id", "event_id", "score",
                                       "azi", "zen"])
sim_pdf = drop_fields(sim_pdf_full, ["azi", "zen", "score", "weight_E2",
                                     "weight_honda", "trueAzi", "trueZen"])

# tdepps setup

In [ ]:
spatial_pdf_args = {"bins": sin_dec_bins, "k": 3, "kent": True}

energy_pdf_args = {"bins": [sin_dec_bins, logE_bins], "gamma": 2.,
                   "fillval": "col", "interpol_log": False,
                   "mc_bg_weights": None,
                   "smooth_sigma": [[0., 0.], [0., 0.]], "logE_asc": False}

time_pdf_args = {"nsig": 4., "sigma_t_min": 2., "sigma_t_max": 30.}

llh_args = {"sob_rel_eps": 0., "sob_abs_eps": 1e-4}

grbllh_1 = LLH.GRBLLH(X=data_pdf, MC=sim_pdf,
                      spatial_pdf_args=spatial_pdf_args,
                      energy_pdf_args=energy_pdf_args,
                      time_pdf_args=time_pdf_args,
                      llh_args=llh_args)

grbllh_2 = LLH.GRBLLH(X=data_pdf, MC=sim_pdf,
                      spatial_pdf_args=spatial_pdf_args,
                      energy_pdf_args=energy_pdf_args,
                      time_pdf_args=time_pdf_args,
                      llh_args=llh_args)

grbllh_3 = LLH.GRBLLH(X=data_pdf, MC=sim_pdf,
                      spatial_pdf_args=spatial_pdf_args,
                      energy_pdf_args=energy_pdf_args,
                      time_pdf_args=time_pdf_args,
                      llh_args=llh_args)

## Test LLH evaluation

In [ ]:
# Create data to evaluate from on data
names = ["timeMJD", "ra", "sinDec", "logE", "sigma"]
types = len(names) * [np.float]
dtype = [(name, typ) for name, typ in zip(names, types)]
X = np.empty((len(on_data)), dtype=dtype)

for name in X.dtype.names:
    X[name] = on_data[name]

In [ ]:
# Approx BG expectation per GRB: 5mHz * dt
nb = 5e-3 * (grb_srcs["dt1"] - grb_srcs["dt0"])

3 sources

In [ ]:
multillh3 = LLH.MultiSampleGRBLLH()
multillh3.add_sample("GRB2013a", grbllh_1)
multillh3.add_sample("GRB2013b", grbllh_2)
multillh3.add_sample("GRB2013c", grbllh_3)

In [ ]:
X_dict = {"GRB2013a": X, "GRB2013b": X, "GRB2013c": X}
args_dict = {"GRB2013a": {"nb": nb, "srcs": grb_srcs},
             "GRB2013b": {"nb": nb, "srcs": grb_srcs},
             "GRB2013c": {"nb": nb, "srcs": grb_srcs}}

In [ ]:
%%time
ns = np.linspace(0, 10, 100)
llh3, grad3 = np.array([multillh.lnllh_ratio(X_dict, nsi, args_dict)
                        for nsi in ns]).T

In [ ]:
plt.plot(ns, llh3)
plt.show()
plt.plot(ns, grad3)
plt.plot(0.5 * (ns[:-1] + ns[1:]), np.diff(llh3) / np.diff(ns), ls="--")
plt.show()

2 sources

In [ ]:
multillh2 = LLH.MultiSampleGRBLLH()
multillh2.add_sample("GRB2013a", grbllh_1)
multillh2.add_sample("GRB2013b", grbllh_2)
X_dict = {"GRB2013a": X, "GRB2013b": X}
args_dict = {"GRB2013a": {"nb": nb, "srcs": grb_srcs},
             "GRB2013b": {"nb": nb, "srcs": grb_srcs}}

In [ ]:
%%time
ns = np.linspace(0, 10, 100)
llh2, grad2 = np.array([multillh.lnllh_ratio(X_dict, nsi, args_dict)
                        for nsi in ns]).T

In [ ]:
plt.plot(ns, llh2)
plt.show()
plt.plot(ns, grad2)
plt.plot(0.5 * (ns[:-1] + ns[1:]), np.diff(llh2) / np.diff(ns), ls="--")
plt.show()

1 source

In [ ]:
multillh1 = LLH.MultiSampleGRBLLH()
multillh1.add_sample("GRB2013a", grbllh_1)
X_dict = {"GRB2013a": X}
args_dict = {"GRB2013a": {"nb": nb, "srcs": grb_srcs}}

In [ ]:
%%time
ns = np.linspace(0, 10, 100)
llh1, grad1 = np.array([multillh.lnllh_ratio(X_dict, nsi, args_dict)
                        for nsi in ns]).T

In [ ]:
plt.plot(ns, llh1)
plt.show()
plt.plot(ns, grad1)
plt.plot(0.5 * (ns[:-1] + ns[1:]), np.diff(llh1) / np.diff(ns), ls="--")
plt.show()

Single sample LLH must be the same as multi LLH for a single added sample

In [ ]:
args = {"nb": nb, "srcs": grb_srcs}

In [ ]:
%%time
ns = np.linspace(0, 10, 100)
llh0, grad0 = np.array([grbllh_1.lnllh_ratio(X, nsi, args)
                        for nsi in ns]).T

In [ ]:
plt.plot(ns, llh0)
plt.plot(ns, llh1, ls="--")
plt.show()
plt.plot(ns, grad0)
plt.plot(0.5 * (ns[:-1] + ns[1:]), np.diff(llh0) / np.diff(ns), ls="--")
plt.plot(ns, grad1, ls=":", lw=1)
plt.show()

# Misc

In [ ]:
# Choose your rate function
rate_func_obj = RateFunc.Sinus1yrConstRateFunction(rndgen)

# Make the injector based on the rate function
bg_rate_inj = BGRateInj.RunlistBGRateInjector(rate_func_obj, goodruns,
                                              random_state=rndgen)

# Fit the injector to make it usable
times = data_pdf["timeMJD"]
rate_func = bg_rate_inj.fit(T=times, x0=None, remove_zero_runs=False)

# Now the evt injector for the other attributes
bg_inj = BGInj.MRichmanBGInjector(random_state=rndgen)

if type(bg_inj) == BGInj.DataBGInjector:
    bg_inj.fit(X=data_pdf)
elif type(bg_inj) == BGInj.MRichmanBGInjector:
    edges = np.vstack(([1., -0.1, 0.], [5., np.pi / 2., np.deg2rad(10.)])).T
    mr_nbins = 10
    all_bins = bg_inj.fit(X=data_pdf, nbins=mr_nbins, minmax=False)

In [ ]:
gamma = 2.
sin_dec_range = spatial_pdf_args["bins"][[0, -1]]
sin_dec_range = [-0.1, 1.]
sig_inj = SigInj.SignalInjector(gamma=gamma, inj_width=np.deg2rad(3),
                                sin_dec_range=sin_dec_range,
                                random_state=rndgen)

sig_inj.fit(MC=sim_pdf, srcs=grb_srcs, exp_names=data_pdf.dtype.names)

print("\nFlux for one injected event is: {:.4f} GeV^-1 cm^-2".format(
    sig_inj.mu2flux(1., per_source=False)))